# Compile Socioeconomic status datasets (neighbourhood × year)

This notebook reads all provided CSV files, harmonizes keys (`neighbourhood_code`, `year`) and produces a single **wide** dataset with all indicators as columns.

Outputs:
- `compiled_neighbourhood_year_wide.parquet`
- `compiled_neighbourhood_year_wide.csv`

In [ ]:
import re
from pathlib import Path
import pandas as pd
import numpy as np


In [ ]:
DATA_DIR = Path("/mnt/data")  # change if needed
csv_files = sorted(DATA_DIR.glob("*.csv"))
csv_files


## Helper functions

In [ ]:
def read_cbs_csv(path: Path) -> pd.DataFrame:
    """Read a CBS-style CSV export (semicolon delimited, decimal comma)."""
    # Some files contain occasional malformed lines. We skip those lines rather than failing.
    return pd.read_csv(
        path,
        sep=";",
        decimal=",",
        engine="python",
        encoding="utf-8-sig",
        on_bad_lines="skip",
    )

def normalize_key_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize neighbourhood code/name + year columns."""
    df = df.copy()

    col_year = next((c for c in df.columns if c.strip().lower() == "perioden"), None)
    col_name = next((c for c in df.columns if c.strip().lower() == "wijken en buurten"), None)
    col_code = next((c for c in df.columns if "regiocode" in c.strip().lower()), None)

    if col_year is None or col_name is None or col_code is None:
        raise ValueError(f"Could not find key columns. Columns were: {list(df.columns)}")

    df = df.rename(columns={col_year: "year", col_name: "neighbourhood", col_code: "neighbourhood_code"})

    df["neighbourhood_code"] = df["neighbourhood_code"].astype(str).str.strip()
    df["neighbourhood"] = df["neighbourhood"].astype(str).str.strip()

    df["year"] = (
        df["year"]
        .astype(str)
        .str.extract(r"(\d{4})", expand=False)
        .astype(float)
        .astype("Int64")
    )

    df = df.dropna(subset=["year"])
    return df

def make_safe_column_names(columns):
    """Create analysis-friendly indicator column names."""
    safe = []
    for c in columns:
        c2 = c.strip()
        c2 = c2.replace("%", "pct")
        c2 = re.sub(r"\s+", "_", c2)
        c2 = re.sub(r"[\(\)\[\]\{\}]", "", c2)
        c2 = re.sub(r"[^0-9A-Za-z_\/\-]", "", c2)
        c2 = c2.replace("/", "__")
        c2 = re.sub(r"__+", "__", c2)
        c2 = re.sub(r"_+", "_", c2)
        safe.append(c2)

    out = []
    seen = {}
    for c in safe:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}__dup{seen[c]}")
    return out


## Read and compile all files

In [ ]:
dfs = []
meta = []

for path in csv_files:
    df = read_cbs_csv(path)
    df = normalize_key_cols(df)

    indicator_cols = [c for c in df.columns if c not in ["neighbourhood_code", "neighbourhood", "year"]]

    safe_names = make_safe_column_names(indicator_cols)
    df = df.rename(columns=dict(zip(indicator_cols, safe_names)))

    meta.append({
        "file": path.name,
        "n_rows": len(df),
        "n_cols": len(df.columns),
        "min_year": int(df["year"].min()),
        "max_year": int(df["year"].max()),
    })
    dfs.append(df)

meta_df = pd.DataFrame(meta).sort_values("file")
meta_df


In [ ]:
# Build a stable neighbourhood name mapping (code -> most common name)
name_map = pd.concat([d[["neighbourhood_code", "neighbourhood"]] for d in dfs], ignore_index=True)
name_map = name_map.dropna().drop_duplicates(subset=["neighbourhood_code", "neighbourhood"])
top_name = (name_map.groupby("neighbourhood_code")["neighbourhood"]
            .agg(lambda s: s.value_counts().index[0])
            .rename("neighbourhood"))

# Merge all indicators (drop neighbourhood name during merges to avoid duplicate column-name issues)
wide = dfs[0].drop(columns=["neighbourhood"])
for i, df in enumerate(dfs[1:], start=1):
    wide = wide.merge(
        df.drop(columns=["neighbourhood"]),
        on=["neighbourhood_code", "year"],
        how="outer",
        suffixes=("", f"__from_{i}"),
    )

# Coalesce columns that represent the same indicator coming from multiple files
from_cols = [c for c in wide.columns if "__from_" in c]
bases = set(c.split("__from_")[0] for c in from_cols)

for base in bases:
    variants = [c for c in wide.columns if c == base or c.startswith(base + "__from_")]
    if base not in wide.columns:
        wide[base] = np.nan
    for v in variants:
        if v == base:
            continue
        wide[base] = wide[base].fillna(wide[v])
    wide = wide.drop(columns=[v for v in variants if v != base])

# Attach neighbourhood names back
wide = wide.merge(top_name.reset_index(), on="neighbourhood_code", how="left")

# Reorder
key_cols = ["neighbourhood_code", "neighbourhood", "year"]
other_cols = [c for c in wide.columns if c not in key_cols]
wide = wide[key_cols + sorted(other_cols)]

wide.shape


In [ ]:
wide[key_cols].isna().sum()


In [ ]:
wide.duplicated(subset=["neighbourhood_code","year"]).sum()


In [ ]:
wide.head()


## Save outputs

In [ ]:
out_csv = DATA_DIR / "compiled_neighbourhood_year_wide.csv"
out_pickle = DATA_DIR / "compiled_neighbourhood_year_wide.pkl"

wide.to_csv(out_csv, index=False)
wide.to_pickle(out_pickle)

out_csv, out_pickle


## Example plots

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
numeric_cols = [c for c in wide.columns if c not in key_cols and pd.api.types.is_numeric_dtype(wide[c])]
numeric_cols[:10], len(numeric_cols)


In [ ]:
example_neighbourhood = wide["neighbourhood"].dropna().iloc[0]
example_indicator = numeric_cols[0]

ts = (wide[wide["neighbourhood"] == example_neighbourhood]
      .sort_values("year")[["year", example_indicator]]
      .dropna())

plt.figure()
plt.plot(ts["year"], ts[example_indicator], marker="o")
plt.title(f"{example_indicator} over time — {example_neighbourhood}")
plt.xlabel("Year")
plt.ylabel(example_indicator)
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
year_to_plot = int(wide["year"].dropna().max())
example_indicator2 = numeric_cols[min(1, len(numeric_cols)-1)]

cross = wide[wide["year"] == year_to_plot][example_indicator2].dropna()

plt.figure()
plt.hist(cross, bins=40)
plt.title(f"Distribution of {example_indicator2} across neighbourhoods ({year_to_plot})")
plt.xlabel(example_indicator2)
plt.ylabel("Count of neighbourhoods")
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
year_to_plot = int(wide["year"].dropna().max())
x = numeric_cols[0]
y = numeric_cols[min(2, len(numeric_cols)-1)]

scatter_df = wide[wide["year"] == year_to_plot][["neighbourhood", x, y]].dropna()

plt.figure()
plt.scatter(scatter_df[x], scatter_df[y], s=10)
plt.title(f"{y} vs {x} ({year_to_plot})")
plt.xlabel(x)
plt.ylabel(y)
plt.grid(True, alpha=0.3)
plt.show()
